In [1]:
import os
import numpy as np

import astropy.table
from astropy.table import Table
import astropy.units as u
from astropy.constants import k_B, m_p



from scipy.interpolate import UnivariateSpline
import hs_atmosphere as atm
from hs_atmosphere import fib

__all__ = ['read_VAL3c_MTW', 'interpolate_atmosphere', 'get_spruit_hs', 'vertical_profile']

SyntaxError: invalid syntax (hs_atmosphere.py, line 23)

In [4]:
#============================================================================
    # Dimensional units in terms of SI
#============================================================================
scales   = {
            'length':         1e2*u.Mm,
            'density':        1e-4*u.kg/u.m**3,
            'velocity':       1e2*u.m/u.s,
            'temperature':    1.0*u.K, 
            'magnetic':       1e-3*u.T #mT
           }
scales['energy density'] = scales['density'] * scales['velocity']**2
scales['time'] = scales['length'] / scales['velocity'] 
scales['mass'] = scales['density'] * scales['length']**3 
scales['force density'] = scales['density'] * scales['velocity'] / \
                       scales['time'] #D momentum/Dt force density balance 
#============================================================================
# physical constants
#============================================================================
physical_constants = {'gamma':      5.0/3.0         , 
                        'mu':         0.602           , 
                      #  'mu0':        asc.mu0         , 
                      #  'boltzmann':  asc.k_B         ,
                      #  'proton_mass':asc.m_p         ,
                        'gravity':    -2.74e2*u.m/u.s/u.s
                        }


In [7]:
print(atm.__name__)

atm.fib(1000)

hs_atmosphere
1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 987 


In [10]:
    l_mpi = False
    rank = 0
    size = 1
#==============================================================================
#set up model parameters
#==============================================================================
local_procs=1
#standard set of logical switches
option_pars = atm.options.set_options(model_pars, l_mpi, l_gdf=True)
#standard conversion to dimensionless units and physical constants
scales, physical_constants = \
    atm.units_const.get_parameters()

#obtain code coordinates and model parameters in astropy units
coords = atm.model_pars.get_coords(model_pars['Nxyz'], u.Quantity(model_pars['xyz']))

#interpolate the hs 1D profiles from empirical data source[s]
empirical_data = atm.hs_atmosphere.read_VAL3c_MTW(mu=physical_constants['mu'])

AttributeError: module 'hs_atmosphere' has no attribute 'options'

In [9]:
model = 'spruit'

local_procs=1
#standard set of logical switches
logical_pars = atm.get_logical(model, l_mpi, l_SI=True, l_gdf=True)

#standard conversion to dimensionless units and physical constants
scales, physical_constants = \
    atm.get_parameters(model, l_mpi, logical_pars, size)

#if 1D or 2D set unused dimensions to 0, and unrequired xyz limits to 1. 
Nxyz = [129,129,129] # 3D grid
xyz_SI = [-1e6,1e6,-1e6,1e6,3.5e4,1.6e6] # xyz limits SI/CGS units    

#obtain code coordinates and model parameters in code units
coords, model_pars = atm.get_model(
                                   model,
                                   Nxyz,
                                   xyz_SI,
                                   scales,
                                   logical_pars
                                  )    

AttributeError: module 'hs_atmosphere' has no attribute 'get_logical'

In [6]:
#identify location of source data files
__files__=''
homedir = os.environ['HOME']
cwd = os.path.dirname(__files__)
#cwd = homedir+'/Dropbox/multi_tube/python/allpapers/'
VAL = os.path.join(cwd, 'hs_model/VALIIIC.dat')
MTW = os.path.join(cwd, 'hs_model/mcwhirter.dat')

filenames = [VAL, MTW]
# uncomment and switch to l_const/l_sqrt/l_linear/l_square as required  
#logical_pars['l_square'] = True 

NameError: name 'logical_pars' is not defined

In [17]:
VAL3c = np.loadtxt(filenames[0])[::-1,:]
VAL3c[:,0] *= 1e3 #km -> m
VAL3c[:,1] *= 1e3 #g/cm^3 -> kg/m^3
VAL3c[:,2] /= 1e1 #dyne/cm^2 -> Pascals(N/m^2)
muTV=4.0/(3*0.74+1+VAL3c[:,6]/VAL3c[:,5])
nzv=VAL3c[:,0].size
MTW =np.loadtxt(filenames[1])[::-1,:]
        MTW[:,0] *= 1e3 #km -> m
        MTW[:,1] *= 1   #Kelvin
        MTW[:,2] /= 1e1 #dyne/cm^2 -> Pascals(N/m^2)
        nzm=MTW[:,0].size
        #Combine both sets into single array
        hdata=np.zeros(nzv+nzm)
        hdata[0:nzv]=VAL3c[:,0]
        hdata[nzv:nzv+nzm]=MTW[:,0]
        hdata /= scales['length']
    
        pdata=np.zeros(nzv+nzm)
        pdata[0:nzv]=VAL3c[:,2]
        pdata[nzv:nzv+nzm]=MTW[:,2]
        pdata /= scales['energy density']
    
        Tdata=np.zeros(nzv+nzm)
        Tdata[0:nzv]=VAL3c[:,3]
        Tdata[nzv:nzv+nzm]=MTW[:,1] 
        Tdata /= scales['temperature']
    
        rdata=np.zeros(nzv+nzm)
        rdata[0:nzv]=VAL3c[:,1]
        rdata /= scales['density']   
        #MTW[:,2], kB, mp and mu are in code units so no rescale needed
        rdata[nzv:nzv+nzm] = MTW[:,2]/physicalconstants['boltzmann']/MTW[:,1] \
                        *physicalconstants['proton_mass']*physicalconstants['mu']
    
        muofT = np.zeros(nzv+nzm) # mean molecular weight
        muofT[0:nzv] = muTV
        muofT[nzv:nzv+nzm] = physicalconstants['mu']
        # interpolate total pressure, temperature and density profiles
        pdata_f = UnivariateSpline(hdata,np.log(pdata),k=1, s=0.25)
        Tdata_f = UnivariateSpline(hdata,np.log(Tdata),k=1, s=0.25)
        rdata_f = UnivariateSpline(hdata,np.log(rdata),k=1, s=0.25)
        #s=0.0 to ensure all points are strictly used for ionisation state
        muofT_f = UnivariateSpline(hdata,np.log(muofT),k=1, s=0.0)
        pdata_i = np.exp(pdata_f(Z))
        Tdata_i = np.exp(Tdata_f(Z))
        rdata_i = np.exp(rdata_f(Z))
        muofT_i = np.exp(muofT_f(Z))

        source_data = [VAL3c,MTW]           

IndentationError: unexpected indent (<ipython-input-17-151f884d1742>, line 7)